When we run many times k-means clustering, we can obtain many results.
To illustrate, if we have 15 samples and want to cluster these 15 samples in clusters, we use k-means clustering.
In unsupervised learning, we do not know before hand the number of clusters that we will obtain. Thus, we need to 
define the number of clusters that we want.
In our example, we want to cluster 15 samples. These 15 samples have many features such as height, width, top position, left position
At the beginning we use only the feature height with k-means clustering. Thus, we obtain the next result
[1., 1., 2., 2., 3., 1., 1., 1., 1., 1., 4., 5., 4., 5., 5.]
We can run k-means clustering with another features such as width and we obtain
[1., 2., 3., 3., 4., 1., 1., 1., 1., 1., 3., 4., 5., 4., 5.]
And so on
[1., 2., 3., 3., 3., 1., 1., 1., 1., 1., 4., 5., 4., 5., 5.]
[1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 4., 5., 5., 5., 5.]

We want to use voting to determine which is the best cluster for each datasamples

At the begining, we determine which are the samples that belong to the same cluster such as
[1., 1., 2., 2., 3., 1., 1., 1., 1., 1., 4., 5., 4., 5., 5.]

[([[0],[1],[5],[6],[7],[8],[9]]), ([[2],[3]]),([[4]]),([[10],[12]]),([[11],[13],[14]])]
The next 
[1., 2., 3., 3., 4., 1., 1., 1., 1., 1., 3., 4., 5., 4., 5.]

[([[0],[5],[6],[7],[8],[9]]), ([[1]]),([[ 2],[ 3],[10]]), ([[ 4],[11],[13]]), ([[12],[14]])]
...
[1., 2., 3., 3., 3., 1., 1., 1., 1., 1., 4., 5., 4., 5., 5.]

[([[0],[5],[6],[7],[8],[9]]),([[1]]),([[2],[3],[4]]),([[10],[12]]),([[11],[13],[14]])]
...
[1., 1., 2., 2., 3., 3., 3., 3., 3., 3., 4., 5., 5., 5., 5.]

[([[0],[1]]),([[2],[3]]),([[4],[5],[6],[7],[8],[9]]),([[10]]),([[11],[12],[13],[14]])]

Next, we want to obtain a pairwise comparison matrix.
The combination [0,1] is repeated 2 times in the clusters
the combination [4,6] is repeated 4 times in the clusters 

the combination 7 and 8 appears 4 times. It means that all the k-means clustering are the same
   0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14 
0  1  2  0  0  0  3  3  3  3  3  0   0   0   0   0   
1  2  1  0  0  0  1  1  1  1  1  0   0   0   0   0   
2  0  0  1  4  1  0  0  0  0  0  1   0   0   0   0       
3  0  0  4  1  1  0  0  0  0  0  1   0   0   0   0   
4  0  1  1  1  1  1  1  1  1  1  0   1   0   1   0     
5  3  1  0  0  1  1  4  4  4  4  0   0   0   0  0  
6  3  1  0  0  1  4  1  4  4  4  0   0   0   0   0 
7  3  0  0  0  1  4  4  1  4  4  0   0   0   0   0 
8  3  1  0  0  1 .....  
9  .....
10
11
12
13
14
15

When the value is 4, it means that all the clusters voted by the same class.
We establish a threshold to decide if a samples belongs to one cluster or another



In [ ]:
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score,v_measure_score
from operator import itemgetter

#The function k_means_clustering obtains many clusters. 
#Each cluster is obtained depending on the selected features.
#For example, we can create a cluster with the feature left.
#We can create another cluster with the features left-height
#We can create another cluster with the features left-height-top


def k_means_clustering(all_features):


    km_scores= []
    km_silhouette = []
    vmeasure_score =[]
    db_score = []
    matrix_results_clustering=[]


    left=[4]     #feature selected to create a cluster
    height=[3]   #feature selected to create a cluster
    left_height=[3,4]  #features selected to create a cluster
    left_height_top=[3,4,5]  #features selected to create a cluster
    left_heigth_width=[3,4,6]  #features selected to create a cluster
    left_height_appearance_width=[2,3,4,6]  #features selected to create a cluster
    left_height_appearance_width_top=[2,3,4,5,6] #features selected to create a cluster
    all_combination=[left,height,left_height,left_height_top,left_heigth_width,left_height_appearance_width,left_height_appearance_width_top] # This array contains all the possible scenarios to obtain many clusters. We only need to add to this array another scenario to be run in a loop.
    number_rows=all_features.shape[0]  #obtain the number of rows
    kmeans_kwargs = {"init": "random","random_state": 0}  #initialize the k-means clustering
    k=5   #we define the number of clusters
    #We choose 5 clusters because the medicine can contain information related to claim, caution, company name, product name, and other
    for i in all_combination:  #this loop helps to iterate and obtain the results for each scenario
        res=all_features[:,i]  #the dataset features contains information related to each picture
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs) #we train each cluster with diferent combination of features
        kmeans.fit(res)  #we fit the model
        preds = kmeans.predict(res)  #we predict the results for each line in the medicine label
        result_k_means_transform=np.zeros(len(preds)) 
        number_cluster_optimal=5
        labels=np.arange(1, number_cluster_optimal+1, 1)
        
        #When we obtain a result from k-means clustering using sklearn, the labels of the result are not in order.
        #It means that the labels can be [5,5,4,4,2,2,1,0] or [4,3,2,4,9,4,0,1]
        #the function transform_label_k_means change the labels depending on the order 
        #The results are the following [0,0,1,1,2,2,3,4] or [0,1,2,0,3,0,4,5]
        #The function takes many parameters suchs preds which is the result obtained from k-means clustering
        #labels: this parameter depends on the number of clusters that we selected to illsutrate
        #if we have 5 clusters, labels=[0,1,2,3,4]
        k_means_result=(transform_label_k_means(preds,labels,result_k_means_transform))
        matrix_results_clustering.append(k_means_result)  #this array concatenates all the results in a matrix
    return matrix_results_clustering 


In [ ]:
def transform_labels(array_element,total,label):
    for j in array_element:
            total[j[0]]=label
    return total
def transform_label_k_means(result_k_means,labels,result_k_means_transform):
    index_sets = [np.argwhere(i==result_k_means) for i in np.unique(result_k_means)]
    #print(index_sets)
    result_k_means_groups=sorted(index_sets, key=itemgetter(0))
   
    aux=1
    for i in result_k_means_groups:
        for labels_element in labels:
            #print(i)
            if (labels_element==aux):
                result_k_means_transform=transform_labels(i,result_k_means_transform,labels_element)
            

        aux=aux+1

   
    return result_k_means_transform

In [ ]:
#this function uses voting to select which is the best clustering for each line in the picture
#Basically, we find the pairwise relationship of each label.
#After findind the pairwise realtionship, we calculate a rank.
#If the relationship in the pairwise relationship is greater than a threshold, we decided if the lines belong to one or other cluster
# there are two parameters pass to this function
#The first parameter is the matrix which contains all the results obtained of many clusters
#We pass the length of the lines to cluster 
def consensus_clustering(matrix_results_clustering,length_row): 
    number_rows=length_row
    matrix_elements_clustering=[]
    
    #the following loop finds and groups lines of the medicine that belongs to the same cluster
    #To illustrate, if we have the cluster [0,0,1,1,2,2,3,4], we obtain a result as the following
    #{[0,1],[2,3],[4,5],[6],[7]}. If we have another reulst such as [0,0,1,2,2,2,3,4],
    #we obtain {[0,1],[1],[2,3,4],[5],[6]}
    # We obtained a vector with the results obtained when we have many result clusters
    
    for result_k_means_element in matrix_results_clustering:
        index_sets = [np.argwhere(i==result_k_means_element) for i in np.unique(result_k_means_element)]
        print(index_sets)
        matrix_elements_clustering.append(index_sets)
        #print(preds)
    
    
    #find consensus clustering
    #The following lines of code find a pairwise relationship of each line in the medicine
    count_pair_coincidences=0  #this variable counts pair wise element in the same cluster
    matrix_pairwise_coincidences=np.zeros((number_rows, number_rows))
    for element_i in range(0,number_rows):
        for element_j in range(0,number_rows):
            count_pair_coincidences=0
            probability=0
            #print("length")
            length_number_of_possibilities_clustering=len(matrix_elements_clustering) #this line determine how many clusters we have depending on the clusters obtained. To illsutrate, if we have a model for left is 1. If we have another cluster left,top is two. If we have another cluster left, heigth, we have three clusters.
            for element in matrix_elements_clustering:
                for element_element in element:
                    
                    if element_i in element_element:
                        if element_j in element_element:
                            count_pair_coincidences=count_pair_coincidences+1

            matrix_pairwise_coincidences[element_i,element_j]=count_pair_coincidences

    #we obtained a pairwise relationship matrix depending on the number of clusters that we obtained
    #we obtained a probability matrix which indicates if a line belongs to each cluster depending on a voting of many result clusters 
    print(matrix_pairwise_coincidences)
    probabilities=matrix_pairwise_coincidences/length_number_of_possibilities_clustering
    print(probabilities)
    
    
    
    #This threshold indicates the final decision to group lines in a cluster
    #we transform the matrix of probabilties to 1 if the values are greater than the threshold
    threshold_define_clusters=0.7

    ones_transform_probabilities=np.zeros((number_rows, number_rows))
    for row in range(0,number_rows):
        for column in range(0,number_rows):
                if probabilities[row][column]>threshold_define_clusters:
                    ones_transform_probabilities[row][column]=1
                
    #print(ones_transform_probabilities)
    
    #we eliminate duplicate rows of the final result
    uniques = np.unique(ones_transform_probabilities,axis=0)
    print(uniques)
    # we obtain a logical and among rows of the matrix to define if two results have elements in common
    length_rows,length_columns=uniques.shape
    array_inter=[]
    array_inter2=[]
    for i in range(0,length_rows):
        array_inter=[]
        for j in range(0,length_rows):
            intersection=np.logical_and(uniques[i],uniques[j])
            
            sum_intersection=sum(intersection)
            if sum_intersection!=0:
                if i!=j:
                   
                    array_inter.append(i)
                    array_inter.append(j)
        array_inter2.append(array_inter)
    #we obtain the rows that contain elements in common in order to join lines
    array_inter3=[]
    for i in array_inter2:
        array_inter3.append(list(np.unique(i)))
    print(array_inter3)
    a=np.unique(array_inter3)
    print(a)
    final_to_join=[]
    for i in a:
        length_eliminate=len(i)
        if (length_eliminate!=0):
            final_to_join.append(i)
    print("final to join")
    print(final_to_join)
    #In the following lines of code, we join all the lines with elements in common
    union_final=[]
    for vector_to_join in final_to_join:
        s=uniques[vector_to_join]
        union=np.logical_or(*s)
        union_final.append(list(union))
    print(union_final)
    #We find the rows in the matrix that were not included in the unions
    flat_list = [item for sublist in final_to_join for item in sublist]    
    print(flat_list)

    all_elements=np.arange(0,length_rows)
    all_elements=all_elements.tolist()
    print(all_elements)

    for i in flat_list:
        print(i)
        if i in all_elements:
                #all_elements=all_elements.tolist()
                all_elements.remove(i)
 
    remaining_vectors=uniques[all_elements]
    print(remaining_vectors)
    print((union_final))
    #we concatenate the results
    matrix_consensus=np.concatenate((remaining_vectors, union_final), axis=0)
    print(matrix_consensus)
    return(matrix_consensus)
